In [ ]:
! pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/ece1786-project

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/ece1786-project


In [ ]:
import transformers
from datasets import Dataset
from transformers import AutoTokenizer
import pandas as pd

In [ ]:
model_checkpoint = "gpt2" #"distilgpt2"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2", use_fast=True)

In [ ]:

df = pd.read_pickle("cocktail_dataset.pkl")  
recipes = []
for i in range(len(df.index)):
  recipe = "RECIPE NAME\n" + df.loc[i, "Name"] + " \n\nRECIPE INGREDIENTS\n"
  skip = False
  for ingredient in df.loc[i, "Ingredients"]:
    # remove asterisks
    ingredient.replace('*', '')
    if "750" in ingredient:
      #skip batch drink recipes
      skip = True
    recipe += ingredient + "\n"
  if skip:
    continue
  recipe += "\nRECIPE INSTRUCTIONS\n" 
  for instruction in  df.loc[i, "Instructions"]:
    if instruction.startswith("*"):
      continue
    recipe += instruction + "\n"
  recipes.append(recipe)

tokenizer.pad_token = tokenizer.eos_token
data = tokenizer(recipes, padding='longest')
data["labels"] = data["input_ids"].copy()
print(data.keys())
dataset = Dataset.from_dict(data)
print(len(dataset))
# print(dataset[5])
print(dataset)

dict_keys(['input_ids', 'attention_mask', 'labels'])
292
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 292
})


In [ ]:
print(len(dataset[5]['input_ids']))

319


In [ ]:
from transformers import AutoModelForCausalLM, GPT2Model
from transformers import Trainer, TrainingArguments
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-recipes",
    evaluation_strategy = "epoch",
    num_train_epochs=50,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=True
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/0dd7bcc7a64e4350d8859c9a2813132fbf6ae591/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_vers

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset
)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 292
  Num Epochs = 50
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1850
  Number of trainable parameters = 124439808


Epoch,Training Loss,Validation Loss
1,0.842600,0.816406
2,0.561100,0.683949
3,0.634800,0.624084
4,0.526300,0.586055
5,0.694700,0.558562
6,0.975500,0.534396
7,0.529300,0.514674
8,0.524300,0.495443
9,0.536800,0.479761
10,0.538000,0.464749


***** Running Evaluation *****
  Num examples = 292
  Batch size = 8
***** Running Evaluation *****
  Num examples = 292
  Batch size = 8
***** Running Evaluation *****
  Num examples = 292
  Batch size = 8
***** Running Evaluation *****
  Num examples = 292
  Batch size = 8
***** Running Evaluation *****
  Num examples = 292
  Batch size = 8
***** Running Evaluation *****
  Num examples = 292
  Batch size = 8
***** Running Evaluation *****
  Num examples = 292
  Batch size = 8
***** Running Evaluation *****
  Num examples = 292
  Batch size = 8
***** Running Evaluation *****
  Num examples = 292
  Batch size = 8
***** Running Evaluation *****
  Num examples = 292
  Batch size = 8
***** Running Evaluation *****
  Num examples = 292
  Batch size = 8
***** Running Evaluation *****
  Num examples = 292
  Batch size = 8
***** Running Evaluation *****
  Num examples = 292
  Batch size = 8
Saving model checkpoint to gpt2-finetuned-recipes/checkpoint-500
Configuration saved in gpt2-finetuned-

TrainOutput(global_step=1850, training_loss=0.45332525818734554, metrics={'train_runtime': 1788.9955, 'train_samples_per_second': 8.161, 'train_steps_per_second': 1.034, 'total_flos': 2376838886400000.0, 'train_loss': 0.45332525818734554, 'epoch': 50.0})

In [ ]:
prompt = "RECIPE NAME\nMagnum Opus"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda:0")

outputs = model.generate(input_ids, do_sample=True, max_length=100,temperature = 0.8, top_p = 1, return_dict_in_generate=True, output_scores=True)
print(tokenizer.batch_decode(outputs.sequences, skip_special_tokens=True)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


RECIPE NAME
Magnum Opus Colada 

RECIPE INGREDIENTS
2 ounces Aurora CBD-infused rum*
1 ounce St. George Spirits vodka
1/4 ounce St-Germain elderflower liqueur (equal parts St-Germain and St-Germain elderflower oils)
Garnish: lemon twist

RECIPE INSTRUCTIONS
Add all ingredients into a shaker with ice and


In [ ]:
trainer.save_model("gpt2model")

Saving model checkpoint to gpt2model
Configuration saved in gpt2model/config.json
Model weights saved in gpt2model/pytorch_model.bin


In [ ]:
model1 = AutoModelForCausalLM.from_pretrained("./gpt2model").to("cuda:0")

loading configuration file ./gpt2model/config.json
Model config GPT2Config {
  "_name_or_path": "./gpt2model",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "use_cache": true,
  "vocab_size": 50257
}

loading we

In [ ]:
prompt1 = "RECIPE NAME\nMagnum Opus"
input_ids1 = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda:0")

outputs1 = model1.generate(input_ids1, do_sample=True, max_length=100, temperature = 0.8, top_p = 1, return_dict_in_generate=True, output_scores=True)
print(tokenizer.batch_decode(outputs1.sequences, skip_special_tokens=True)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


RECIPE NAME
Magnum Opus Domaine 

RECIPE INGREDIENTS
1 small creme de menthe
1 ounce creme de cacao
2 ounces white rum
1 ounce peach schnapps
1 ounce lemon juice, freshly squeezed
Garnish: brandied cherry
Garnish: orange slices

RECIPE INSTRUCTIONS
Add the creme de menthe, cacao, white rum, peach schnapps
